# 使用 DuckDB UDF 加速 Pandas

[1] 使用 DuckDB UDF 加速 Pandas : https://zhuanlan.zhihu.com/p/646788236

最近需要对一批商品数据进行分析，加上前后各种数据清洗，

初版程序使用 Pandas 运行耗时几个小时，中间耗时最长的是各种 groupby 后再计算。

经过逐步优化，耗时减少到不到半个小时，随记录下中间的一些优化过程。

其中有一个 groupby 是计算每个商品的回归系数，Mock 数据如下：

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm # pip install statsmodels

"""
pip install statsmodels
pip install joblib
pip install duckdb
pip install chdb # clickhouse-local (mac, linux)
"""

def mock(i: int):
    nobs = 10
    X = np.random.random((nobs, 2))
    beta = [1, .1, .5]
    e = np.random.random(nobs)
    y = np.dot(sm.add_constant(X), beta) + e
    return pd.DataFrame(X, columns=["x1", "x2"]).assign(y=y, key=f"c{i:0>4}").filter(["key", "x1", "x2", "y"])


df = pd.concat([mock(i) for i in range(10000)])

商品 c1, c2, ...，自变量 x1 、x2 ，因变量 y，分别计算每个商品的回归系数：

In [2]:
df

,key,x1,x2,y
0,c0000,0.320411,0.914019,1.758490
1,c0000,0.421961,0.000717,1.095975
2,c0000,0.472938,0.205749,1.981890
3,c0000,0.416125,0.215834,1.442362
4,c0000,0.228299,0.833621,2.295887
...,...,...,...,...
5,c9999,0.708642,0.019811,2.017855
6,c9999,0.466480,0.729859,2.204831
7,c9999,0.187594,0.019971,1.350019
8,c9999,0.768211,0.467250,1.366468


## Pandas 实现

### 版本一

最直接的写法是 groupby.apply：

In [3]:
def ols1(d):
    X = sm.add_constant(d[["x1", "x2"]])
    y = d["y"]
    res = sm.OLS(y, X).fit()
    return res.params

%timeit df.groupby(["key"]).apply(ols1)

12.1 s ± 659 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


统计耗时 14.8s 左右：

```
14.8 s ± 249 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
```

### 版本二

考虑到上面数据通过 apply 返回合并为 DataFrame 会比较慢，做一下改版：

In [4]:
def ols2(d):
    X = sm.add_constant(d[["x1", "x2"]].to_numpy())
    y = d["y"].to_numpy()
    res = sm.OLS(y, X).fit()
    return res.params

%timeit df.groupby(["key"]).apply(ols2)

4.04 s ± 99.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


耗时统计 5.6s 左右，优化效果很明显：

```
5.6 s ± 230 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
```

### 版本三

众所周知，groupby.apply 并没有并行执行，再写一下并行版本进一步进行优化：

In [7]:
from joblib import Parallel, delayed # pip install joblib

def ols3(key, d):
    X = sm.add_constant(d[["x1", "x2"]].to_numpy())
    y = d["y"].to_numpy()
    res = sm.OLS(y, X).fit()
    return np.append([key], res.params)
    
# %%timeit
grouped = df.groupby(["key"])
results = Parallel(n_jobs=-1)(delayed(ols3)(key, group) for key, group in grouped)
results_3_df = pd.DataFrame(results, columns=["key", "const", "x1", "x2"])

耗时统计 2.1s 左右，又优化一大截：
```
2.1 s ± 33.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
```
如果不将结果转换为 DataFrame 是 `1.81s` ：

```
1.81 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
```

## DuckDB 实现

最近使用 DudkDB 处理数据比较多，

与 Python 语言交互时，我们可以从 python 函数中创建一个 DuckDB 用户定义函数（UDF），这样它就可以在 SQL 查询中使用。

这样定义的函数，由数据库调度运行，看下是否能据此优化我们的代码。

### 版本四

首先定义一个回归函数，然后注册给 duckdb：

In [41]:
import duckdb # pip install duckdb
from contextlib import suppress

# df_2 = pd.concat([mock(i) for i in range(10000)])


def ols4(x: list, y: list) -> list[float]:
    X = sm.add_constant(np.array([[r["x1"], r["x2"]] for r in x]))
    res = sm.OLS(y, X).fit()
    return res.params

with suppress(Exception):
    duckdb.remove_function("ols4")
    duckdb.remove_function("ols4")

duckdb.create_function("ols4", ols4)

In [42]:
duckdb.query("select * from df")

┌─────────┬─────────────────────┬───────────────────────┬────────────────────┐
│   key   │         x1          │          x2           │         y          │
│ varchar │       double        │        double         │       double       │
├─────────┼─────────────────────┼───────────────────────┼────────────────────┤
│ c0000   │  0.3204107883503745 │    0.9140187586935561 │ 1.7584898315489523 │
│ c0000   │  0.4219612972142287 │ 0.0007167400957338588 │ 1.0959746259667127 │
│ c0000   │  0.4729384994108561 │   0.20574929220201954 │  1.981889733870671 │
│ c0000   │  0.4161245922765133 │    0.2158336364040867 │ 1.4423616588721528 │
│ c0000   │   0.228299346382541 │    0.8336210600307845 │ 2.2958868832363923 │
│ c0000   │  0.8873711143592423 │    0.3303355527019023 │ 1.3416064764622262 │
│ c0000   │  0.6748109360980198 │    0.0455976433142572 │ 1.6631172022242442 │
│ c0000   │  0.9041066077178131 │   0.04387839507289515 │ 1.5771440524066658 │
│ c0000   │  0.3077707390241732 │    0.8566586038008

这样我们就可以在 SQL 中直接调用，执行测试：

In [58]:
sql = """
with tmp as(
    select key, ols4(list((x1, x2)), list(y)) as coef
    from df
    group by all
)
select key, coef[1] as const, coef[2] as x1, coef[3] as x2
from tmp
order by all
"""
duckdb.sql(sql).df()

# %timeit duckdb.sql(sql).df() # 运行error ？ .df() error？
%timeit duckdb.sql(sql).fetch_df() # 运行error ？ .df() error？

AttributeError: This relation does not contain a column by the name of 'fetch_df'

耗时为 2.9s，看上去还没有上面 Python 并行化版本效率高：

```
2.9 s ± 26.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
```

### 版本五

猜测 *.df() 转 DataFrame 格式耗时比较久，如果不进行格式转换，直接运行：

In [48]:
%timeit duckdb.sql(sql)

811 µs ± 63.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


耗时仅有 825 µs，非常 amazing！

```
825 µs ± 5.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
```

### 版本六

注意到上面 SQL 中为 select * from df，我们查询的是 DataFrame。

如果我们直接查询的是 DuckDB 的表，应该还会进一步减少耗时。例如将 df 存为表：

In [29]:
sql = """
create or replace table example as
select * from df
"""
duckdb.sql(sql)

然后，重新执行计算：

In [30]:
sql = """
with tmp as (
    select key, ols4(list((x1, x2)), list(y)) as coef
    from example
    group by all
)
select key, coef[1] as const, coef[2] as x1, coef[3] as x2
from tmp
order by all
"""
%timeit duckdb.sql(sql)
# duckdb.sql(sql)

96 µs ± 2.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


统计耗时仅有 131 µs：

```
131 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
```

## 总结

回顾下一路下来的优化：

版本	| 说明	| 耗时
---|---|---
版本一	| Pandas 直接 apply	| 14.8s
版本二	|numpy 优化		| 5.6s
版本三	|joblib 并行		| 2.1s
版本四	|DuckDB UDF 处理和输出 DataFrame		| 2.9s
版本五	|DuckDB UDF 处理 DataFrame		| 825µs
版本六	|DuckDB UDF 直接处理表		| 131µs

其实上述比较并不严谨，它们输出的结果格式并不统一（numpy/dataframe/duckdb 等），

而不同技术栈有不同契合的输入/输出上下文。

可以看到版本四相比版本三其实还要慢一点，测试用例只是保证了其在本技术栈上代码和逻辑上都尽可能简洁直观。

对于版本六，如果将结果输出为 DuckDB 的表，加上 IO，也需要花费 2s 多，
如果使用 CTE，直接使用计算结果，进一步计算诸如最大系数之类的，耗时会更少：


```sql
with tmp as (
    select key, ols4(list((x1, x2)), list(y)) as coef
    from example
    group by all
)
select max(coef[1]) from tmp
```

因此，我们也不用标题党的说，通过 DuckDB 将 Pandas 代码优化了多少多少倍。

实践上，如果我们上下游数据处理都是通过 DuckDB，对于复杂的运算在 SQL 中不好实现，

我们可以通过 Python 来实现，这样我们就可以利用 DuckDB 高性能的同时，也能使用 Python 丰富的生态，达到两者兼顾的目的，大大提升我们分析数据的效率。

即使你日常中更多的是使用 Pandas/Polars 等，上例中看到 DuckDB 和 DataFrame 交互是非常方便的，也可以借用 DuckDB 来优化我们的代码，算是一种不错的选择。